In [1]:
import pandas as pd
import sqlite3 as lite

def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
        # engine = sa.create_engine('sqlite:///' + db_file)  # connection via sqlalchemy
        # conn = engine.connect()
    except Error as e:
        print(e)
    return conn


conn = create_connection("CVEfixes.db")

In [2]:
import pandas as pd

query = """
    SELECT *
    FROM file_change
"""

df = pd.read_sql_query(query, conn)

df.head()

,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,code_after,code_before,nloc,complexity,token_count,programming_language
0,100562011220946,2864e767053317538feafa815046fff89e5a16be,ChangeLog,ChangeLog,ChangeLog,ModificationType.MODIFY,"@@ -1,3 +1,55 @@\n+1999-11-09 Ulrich Drepper ...","{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,nan,nan,nan,Batchfile
1,7461293048740,2864e767053317538feafa815046fff89e5a16be,dl-load.c,elf/dl-load.c,elf/dl-load.c,ModificationType.MODIFY,"@@ -149,21 +149,31 @@ local_strdup (const char...","{'added': [(152, ' const char *const start = ...",28,10,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,952.0,260.0,6592.0,C
2,169254254633061,2864e767053317538feafa815046fff89e5a16be,search.texi,manual/search.texi,manual/search.texi,ModificationType.MODIFY,"@@ -475,9 +475,11 @@ elements.\n To remove a s...","{'added': [(478, 'and a pointer to the parent ...",5,3,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...",nan,nan,nan,TeX
3,253444729592566,2864e767053317538feafa815046fff89e5a16be,syslog.c,misc/syslog.c,misc/syslog.c,ModificationType.MODIFY,"@@ -177,10 +177,14 @@ vsyslog(pri, fmt, ap)\n ...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...",229.0,42.0,1240.0,C
4,197890948684264,2864e767053317538feafa815046fff89e5a16be,Makefile,posix/Makefile,posix/Makefile,ModificationType.MODIFY,"@@ -57,7 +57,7 @@ include ../Makeconfig\n \n a...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...",nan,nan,nan,Shell


In [6]:
df.programming_language.value_counts()

programming_language
PHP                 5841
C                   5214
unknown             3792
JavaScript          2004
C++                 1720
Markdown            1495
Python              1208
Java                1187
Ruby                1021
HTML                 870
TypeScript           811
Go                   668
Shell                667
SQL                  347
C#                   283
Batchfile            276
CoffeeScript         268
Objective-C          254
Perl                 228
CSS                  220
Scala                194
Lua                  181
Rust                 118
TeX                   91
Haskell               89
PowerShell            68
Jupyter Notebook      63
Swift                 54
Matlab                43
R                     20
Erlang                 9
None                   5
Name: count, dtype: int64

In [8]:
df = df[['filename', 'programming_language', 'code_before', 'code_after',
         'diff_parsed', 'num_lines_added', 'num_lines_deleted', 'nloc', 'token_count', 'complexity',
         'file_change_id', 'hash', 'change_type', 'old_path', 'new_path']]

df = df.rename(columns={
    'filename': 'file_name',
    'old_path': 'old_file_path',
    'new_path': 'new_file_path',
    'diff_parsed': 'diff',
    'nloc': 'num_lines_in_file',
    'token_count': 'num_tokens_in_file',  
})

df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,file_change_id,hash,change_type,old_file_path,new_file_path
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,100562011220946,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,7461293048740,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,169254254633061,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,253444729592566,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,197890948684264,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile


In [10]:
query = """
    SELECT *
    FROM commits
"""

# Using pandas to read data from the database and store it into a DataFrame
df1 = pd.read_sql_query(query, conn)

df1 = df1[[
    'hash', 'repo_url', 'author', 'committer', 'msg', 'merge'
]]
df1 = df1.rename(columns={
    'msg': 'commit_message',
})

merged_df = df.merge(df1, on='hash', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,file_change_id,hash,change_type,old_file_path,new_file_path,repo_url,author,committer,commit_message,merge
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,100562011220946,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,7461293048740,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,169254254633061,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,253444729592566,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,197890948684264,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False


In [12]:
query = """
    SELECT *
    FROM fixes
"""

# Using pandas to read data from the database and store it into a DataFrame
df3 = pd.read_sql_query(query, conn)

df3 = df3[[
    'cve_id', 'hash'
]]

# df2 = df2.rename(columns={
#     'stars_count': 'repo_stars_count',
# })

merged_df = merged_df.merge(df3, on='hash', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,hash,change_type,old_file_path,new_file_path,repo_url,author,committer,commit_message,merge,cve_id
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199


In [160]:
query = """
    SELECT *
    FROM cwe_classification
"""

# Using pandas to read data from the database and store it into a DataFrame
df4 = pd.read_sql_query(query, conn)

df4 = df4.groupby('cve_id')['cwe_id'].apply(list).reset_index()

merged_df = merged_df.merge(df4, on='cve_id', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable,cwe_id
0,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,241941308837203,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,True,[CWE-252]
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,235368100398254,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",51,22,349,True,[CWE-252]
2,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,161060042440601,_dl_dst_count,"_dl_dst_count( const char * name , int is_path)","['name', 'is_path']","_dl_dst_count (const char *name, int is_path)\...",22,13,199,True,[CWE-252]
3,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,205987001054463,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,False,[CWE-252]
4,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,38197852149195,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",57,28,399,False,[CWE-252]


In [14]:
query = """
    SELECT *
    FROM method_change
"""

# Using pandas to read data from the database and store it into a DataFrame
df6 = pd.read_sql_query(query, conn)

df6 = df6[[
    'file_change_id', 'method_change_id', 'name', 'signature', 'parameters', 'code', 'nloc', 'complexity', 'token_count', 'before_change'
]]

df6 = df6.rename(columns={
    'name': 'method_name',
    'signature': 'method_signature', 
    'parameters': 'method_parameters',
    'code': 'method_code',
    'nloc': 'num_lines_in_method',
    'complexity': 'method_complexity',
    'token_count': 'num_tokens_in_method',
    'before_change': 'vulnerable'
})

In [15]:
merged_df = merged_df.merge(df6, on='file_change_id', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,cve_id,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable
0,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CVE-1999-0199,241941308837203,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,True
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CVE-1999-0199,235368100398254,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",51,22,349,True
2,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CVE-1999-0199,161060042440601,_dl_dst_count,"_dl_dst_count( const char * name , int is_path)","['name', 'is_path']","_dl_dst_count (const char *name, int is_path)\...",22,13,199,True
3,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CVE-1999-0199,205987001054463,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,False
4,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CVE-1999-0199,38197852149195,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",57,28,399,False


In [18]:
merged_df.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'method_change_id',
       'method_name', 'method_signature', 'method_parameters', 'method_code',
       'num_lines_in_method', 'method_complexity', 'num_tokens_in_method',
       'vulnerable'],
      dtype='object')

In [82]:
merged_df.programming_language.value_counts()

programming_language
JavaScript      46728
PHP             17965
C               13840
unknown         13635
C++              4393
Python           4201
Java             3855
Ruby             3056
Go               2274
TypeScript       1193
C#                942
Objective-C       571
HTML              298
Scala             294
Rust              254
Swift             118
Lua                97
CoffeeScript       80
Shell              69
SQL                62
Perl               48
PowerShell         47
Matlab             39
Markdown           29
TeX                10
Haskell             6
Erlang              4
Batchfile           2
None                1
Name: count, dtype: int64

In [170]:
me_py = merged_df[merged_df.programming_language == 'Python']
me_py.shape

(4201, 31)

In [172]:
me_py.to_csv('CVEFixes_python.csv', index=False)

In [173]:
me_c = merged_df[merged_df.programming_language == 'C']
me_c.shape

(13840, 31)

In [174]:
me_c.to_csv('CVEFixes_c.csv', index=False)

In [175]:
me_ja = merged_df[merged_df.programming_language == 'Java']
me_ja.shape

(3855, 31)

In [176]:
me_ja.to_csv('CVEFixes_java.csv', index=False)

In [80]:
merged_df[merged_df['file_change_id']=='7461293048740'].file_change_id

0    7461293048740
1    7461293048740
2    7461293048740
3    7461293048740
4    7461293048740
5    7461293048740
Name: file_change_id, dtype: object

In [36]:
print(merged_df['file_change_id'].iloc[1])

7461293048740


In [38]:
me1 = merged_df.drop(['committer', 'author', 'file_change_id', 'hash'], axis=1)
me1.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'hash', 'change_type',
       'old_file_path', 'new_file_path', 'repo_url', 'author', 'committer',
       'commit_message', 'merge', 'cve_id', 'method_change_id', 'method_name',
       'method_signature', 'method_parameters', 'method_code',
       'num_lines_in_method', 'method_complexity', 'num_tokens_in_method',
       'vulnerable'],
      dtype='object')

In [179]:
me_py = pd.read_csv('CVEFixes_python.csv')

In [184]:
me_py.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable,cwe_id
0,Expressions.py,Python,##############################################...,##############################################...,"{'added': [(20, 'import warnings'), (78, ''), ...",9,0,300.0,1758.0,72.0,...,245361968005491,boboAwareZopeTraverse,"boboAwareZopeTraverse( object , path_items , e...","['object', 'path_items', 'econtext']","def boboAwareZopeTraverse(object, path_items, ...",18,5,106,False,['CWE-22']
1,expression.py,Python,"""""""``chameleon.tales`` expressions.""""""\n\nfrom...","""""""``chameleon.tales`` expressions.""""""\n\nimpo...","{'added': [(3, 'import warnings'), (65, ''), (...",10,1,162.0,930.0,28.0,...,174839916442752,traverse,"traverse( cls , base , request , path_items )","['cls', 'base', 'request', 'path_items']","def traverse(cls, base, request, path_item...",11,3,72,True,['CWE-22']
2,expression.py,Python,"""""""``chameleon.tales`` expressions.""""""\n\nfrom...","""""""``chameleon.tales`` expressions.""""""\n\nimpo...","{'added': [(3, 'import warnings'), (65, ''), (...",10,1,162.0,930.0,28.0,...,148047883253798,traverse,"traverse( cls , base , request , path_items )","['cls', 'base', 'request', 'path_items']","def traverse(cls, base, request, path_item...",17,5,99,False,['CWE-22']
3,testChameleonTalesExpressions.py,Python,from ..expression import getEngine\nfrom . imp...,import unittest\n\nfrom ..expression import ge...,"{'added': [(1, 'import unittest'), (2, ''), (2...",7,0,16.0,87.0,3.0,...,201297513086655,test_underscore_traversal,test_underscore_traversal( self ),['self'],def test_underscore_traversal(self):\n ...,2,1,6,False,['CWE-22']
4,testExpressions.py,Python,import unittest\n\nfrom AccessControl import s...,import unittest\nimport warnings\n\nfrom Acces...,"{'added': [(2, 'import warnings'), (5, 'from z...",20,1,258.0,1808.0,55.0,...,211808858382861,test_evaluate_dict_key_as_underscore,test_evaluate_dict_key_as_underscore( self ),['self'],def test_evaluate_dict_key_as_underscore(s...,3,1,25,True,['CWE-22']


In [190]:
me_py['code_before'].iloc[1]==me_py['code_before'].iloc[2]

True

In [186]:
me_py.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'method_change_id',
       'method_name', 'method_signature', 'method_parameters', 'method_code',
       'num_lines_in_method', 'method_complexity', 'num_tokens_in_method',
       'vulnerable', 'cwe_id'],
      dtype='object')

In [188]:
me_py.vulnerable.value_counts()

vulnerable
False    2604
True     1597
Name: count, dtype: int64

In [138]:
me_py.file_change_id

0       247236981611556
1       160866737889490
2       160866737889490
3        70866476438672
4       171815549949841
             ...       
4196    120034951100030
4197    120034951100030
4198    229517810532068
4199    229517810532068
4200    229517810532068
Name: file_change_id, Length: 4201, dtype: int64

In [292]:
me_py.iloc[1]

file_name                                                   expression.py
programming_language                                               Python
code_before             """``chameleon.tales`` expressions."""\n\nfrom...
code_after              """``chameleon.tales`` expressions."""\n\nimpo...
diff                    {'added': [(3, 'import warnings'), (65, ''), (...
num_lines_added                                                        10
num_lines_deleted                                                       1
num_lines_in_file                                                   162.0
num_tokens_in_file                                                  930.0
complexity                                                           28.0
file_change_id                                            160866737889490
hash                             1f8456bf1f908ea46012537d52bd7e752a532c91
change_type                                       ModificationType.MODIFY
old_file_path                    src/P

In [218]:
me_py[me_py['file_change_id']==171815549949841].code_after.iloc[0] == me_py[me_py['file_change_id']==171815549949841].code_after.iloc[1]

True

In [312]:
filtered_df = me_py.groupby('file_change_id').filter(lambda x: len(x) >= 2)
filtered_df = filtered_df.groupby(['file_change_id', 'method_name']).filter(lambda x: len(x) >= 2)

def process_group(group):
    if len(group) >= 2 and group['vulnerable'].nunique() == 2:
        function_before = group[group['vulnerable']].iloc[0]  # Vulnerable code
        function_after = group[~group['vulnerable']].iloc[0]  # Non-vulnerable code

        return pd.Series({
            'file_name': function_before['file_name'],
            'method_name': function_before['method_name'],
            'code_before': function_before['code_before'],
            'code_after': function_before['code_after'],
            'func_before': function_before['method_code'],
            'func_after': function_after['method_code'],
            'diff': function_before['diff'],
            'num_lines_added': function_before['num_lines_added'],
            'num_lines_deleted': function_before['num_lines_deleted'],
            'num_lines_in_file': function_before['num_lines_in_file'],
            'num_tokens_in_file': function_before['num_tokens_in_file'],
            'repo': function_before['repo_url'],
            'cve_id': function_before['cve_id'],
            'cwe_id': function_before['cwe_id'],
        })

# Apply the function to each group and drop any None results
result_df = filtered_df.groupby(['file_change_id', 'method_name']).apply(process_group).dropna().reset_index(drop=True)

In [313]:
result_df.shape

(1237, 14)

In [314]:
result_df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
0,views.py,create_access,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,"def create_access(request, topic_id):\n top...","def create_access(request, topic_id):\n top...","{'added': [(13, 'from spirit.core.conf import ...",17.0,14.0,171.0,1093.0,https://github.com/nitely/spirit,CVE-2022-0869,['CWE-601']
1,views.py,delete_access,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,"def delete_access(request, pk):\n topic_pri...","def delete_access(request, pk):\n topic_pri...","{'added': [(13, 'from spirit.core.conf import ...",17.0,14.0,171.0,1093.0,https://github.com/nitely/spirit,CVE-2022-0869,['CWE-601']
2,views.py,join_in,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,"def join_in(request, topic_id):\n # todo: r...","def join_in(request, topic_id):\n # todo: r...","{'added': [(13, 'from spirit.core.conf import ...",17.0,14.0,171.0,1093.0,https://github.com/nitely/spirit,CVE-2022-0869,['CWE-601']
3,views.py,publish,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,# -*- coding: utf-8 -*-\n\nfrom django.contrib...,"def publish(request, user_id=None):\n initi...","def publish(request, user_id=None):\n initi...","{'added': [(13, 'from spirit.core.conf import ...",17.0,14.0,171.0,1093.0,https://github.com/nitely/spirit,CVE-2022-0869,['CWE-601']
4,test_keystoneclient.py,test_user_create_update_delete,# vim: tabstop=4 shiftwidth=4 softtabstop=4\n\...,# vim: tabstop=4 shiftwidth=4 softtabstop=4\n\...,def test_user_create_update_delete(self):\...,def test_user_create_update_delete(self):\...,"{'added': [(312, ' def test_disable_user_in...",19.0,2.0,700.0,5991.0,https://github.com/openstack/keystone,CVE-2012-3426,['CWE-264']


In [336]:
import ast
# result_df.cwe_id = result_df.cwe_id.apply(ast.literal_eval)

def remove_duplicates(lst):
    return list(set(lst))

result_df.cwe_id = result_df.cwe_id.apply(remove_duplicates)

In [342]:
cwe_id_attention = [918, 352, 79, 20, 22]

result_df = result_df[result_df['cwe_id'].apply(lambda x: any(f'CWE-{i}' in x for i in cwe_id_attention))]
result_df.shape

(455, 14)

In [344]:
result_df.to_csv('CVEFixes_python_5.csv', index=False)

In [340]:
from collections import Counter

counters = Counter()

for sublist in result_df['cwe_id']:
    counters.update(sublist)

# Display the counts for each element
counters

Counter({'CWE-918': 148,
         'CWE-352': 121,
         'CWE-444': 100,
         'CWE-601': 90,
         'CWE-79': 73,
         'NVD-CWE-noinfo': 68,
         'CWE-20': 66,
         'CWE-400': 65,
         'CWE-22': 61,
         'CWE-94': 41,
         'CWE-200': 36,
         'CWE-264': 28,
         'CWE-89': 27,
         'CWE-287': 24,
         'CWE-502': 23,
         'CWE-78': 20,
         'CWE-312': 18,
         'CWE-21': 18,
         'CWE-59': 17,
         'CWE-125': 17,
         'CWE-611': 13,
         'CWE-843': 13,
         'CWE-346': 12,
         'CWE-863': 12,
         'CWE-74': 11,
         'CWE-295': 11,
         'CWE-285': 11,
         'CWE-119': 8,
         'CWE-367': 7,
         'CWE-330': 6,
         'CWE-203': 6,
         'CWE-269': 6,
         'CWE-770': 6,
         'NVD-CWE-Other': 6,
         'CWE-93': 6,
         'CWE-1333': 5,
         'CWE-668': 5,
         'CWE-331': 4,
         'CWE-697': 4,
         'CWE-613': 4,
         'CWE-116': 3,
         'CWE-684': 3,


In [250]:
for i in range(result_df.shape[0]):
    if not result_df.function_before.iloc[i] in result_df.code_before.iloc[i]:
        print(i)